# Moving and Rotating a Molecular Cluster:
**Task:** 
1. Move/rotate a Molecule/Atom one by one systematically
2. Move/Rotate randomly

## Water molecule ($H_{2}O$):

Cartesian coordinates [Angstrom]:

    
    O    0.00000    0.00000    0.00000
    H    0.58708    0.75754    0.00000
    H    0.58708   -0.75754    0.00000


> **_NOTE:_**   
> `translate` and `rotate` methods return a brand new Molecule object

In [1]:
# crating the path (PYTHONPATH) to our module.
# assuming that our 'src' directory is out ('..') of our current directory 
import os
import sys
module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
# importing de Molecule Class
from src.base_molecule import Molecule, Cluster

In [3]:
# creating a Molecular object. You can use a list, a dictionary 
# (the key MUST be "atoms") or another Molecule object (see notebook: 01)

water_molecule=[("O", 0, 0, 0), ("H", 0.58708, 0.75754, 0), ("H", -0.58708, 0.75754, 0)]

water = Molecule(water_molecule)

print(water)

	3
-- charge= 0 and multiplicity= 1 --
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000



In [4]:
# Let's create a Cluster adding two more water molecules using 
# the same cartesian coordinates. 
# Warning! check the number of molecules in this cluster

water3 = Cluster(water_molecule, 2 * water)

print(water3.total_molecules)
print(water3)

2
Cluster of 2 moleculesand 9 total atoms
molecule 0 (with 3 atoms):
     -> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     -> charge: 0
     -> multiplicity: 1
molecule 1 (with 6 atoms):
     -> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0), ('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     -> charge: 0
     -> multiplicity: 1



>### NOTE:
>Creating a `Molecule` object means that its atoms are FIXED and 
>those atoms **can NOT** be MOVED or ROTATED.
>
>On the other hand, creating a `Cluster` object means their individual
>molecules **can** be MOVED or ROTATED

In [5]:
# doing addition (+) or multiplication (*) for Molecule objects
# create Molecule objects with FIXED atoms
print('object type: ', type(water).__name__)

# creating a cluster of THREE (3) water molecules, we want 3 molecules to
# move or rotate
water3 = Cluster(water_molecule, 2 * Cluster(water))

print('total molecules: ', water3.total_molecules)
print(water3)

# print XYZ format
print(water3.xyz)

object type:  Molecule
total molecules:  3
Cluster of 3 moleculesand 9 total atoms
molecule 0 (with 3 atoms):
     -> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     -> charge: 0
     -> multiplicity: 1
molecule 1 (with 3 atoms):
     -> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     -> charge: 0
     -> multiplicity: 1
molecule 2 (with 3 atoms):
     -> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     -> charge: 0
     -> multiplicity: 1

	9
-- charge= 0 and multiplicity= 1 --
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.5870

In [6]:
# let's create a cluster with five (5) water molecules and one atom (Iodine)
iodine =[("I", 0, 0, 0)]

w5_I = Cluster(
    iodine,
    water,
    water,
    water,
    water,
    water,
)

# or in a much simpler way (all equivalent)

# w5_I = Cluster(iodine, 5 * Cluster(water_molecule))
# w5_I = Cluster(iodine, 5 * Cluster(water))
# w5_I = Cluster(Molecule(iodine) + 5 * Cluster(water))
# w5_I = Cluster(Molecule(iodine), 5 * Cluster(water))
# w5_I = Cluster(Cluster(iodine) + 5 * Cluster(water))
# w5_I = Cluster(Cluster(iodine), 5 * Cluster(water))
# w5_I = Cluster(iodine) + 5 * Cluster(water_molecule)
# w5_I = Cluster(iodine, water_molecule, water, 3 * Cluster(water_molecule))
# w5_I = Molecule(iodine) + 5 * Cluster(water_molecule)

# we prefer this way 
w5_I = Cluster(Molecule(iodine) + 5 * Cluster(water))

# ALWAYS, you have SIX (6) individual molecules/atoms in your 'w5_I' cluster
print(f"Number of individual molecules/atoms: {w5_I.total_molecules}")
print(w5_I)

Number of individual molecules/atoms: 6
Cluster of 6 moleculesand 16 total atoms
molecule 0 (with 3 atoms):
     -> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     -> charge: 0
     -> multiplicity: 1
molecule 1 (with 3 atoms):
     -> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     -> charge: 0
     -> multiplicity: 1
molecule 2 (with 3 atoms):
     -> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     -> charge: 0
     -> multiplicity: 1
molecule 3 (with 3 atoms):
     -> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     -> charge: 0
     -> multiplicity: 1
molecule 4 (with 3 atoms):
     -> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     -> charge: 0
     -> multiplicity: 1
molecule 5 (with 1 atoms):
     -> atoms: [('I', 0, 0, 0)]
     -> charge: 0
     -> multiplicity: 1



In [7]:
# if we want an accurate orden in our cluster, we should use COMMA 
# for example, Iodine as the first molecule in our cluster
w5_I = Cluster(Molecule(iodine), 5 * Cluster(water))

print(w5_I.xyz)

	16
-- charge= 0 and multiplicity= 1 --
I     	     0.00000000	     0.00000000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000



In [8]:
# asking for its properties
print(f"\nNumber of individual molecules/atoms: \n{w5_I.total_molecules}")
print(f"\nTotal number of atoms: \n{w5_I._total_atoms}")
print(f"\nAtomic symbols: \n{w5_I.symbols}")
print(f"\nIndividual atomic masses: \n{w5_I.atomic_masses}")
print(f"\nMolecular mass: \n{w5_I.total_mass}")
print(f"\nCluster dict: \n{w5_I.cluster_dictionary}")
print(f"\nCoordinates: \n{w5_I.atoms}")
print(f"\nCartesian coordinates: \n{w5_I.coordinates}")


Number of individual molecules/atoms: 
6

Total number of atoms: 
16

Atomic symbols: 
['I', 'O', 'H', 'H', 'O', 'H', 'H', 'O', 'H', 'H', 'O', 'H', 'H', 'O', 'H', 'H']

Individual atomic masses: 
[126.9045, 15.999, 1.008, 1.008, 15.999, 1.008, 1.008, 15.999, 1.008, 1.008, 15.999, 1.008, 1.008, 15.999, 1.008, 1.008]

Molecular mass: 
216.9795000000001

Cluster dict: 
{0: {'atoms': [('I', 0, 0, 0)], 'charge': 0, 'multiplicity': 1}, 1: {'atoms': [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)], 'charge': 0, 'multiplicity': 1}, 2: {'atoms': [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)], 'charge': 0, 'multiplicity': 1}, 3: {'atoms': [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)], 'charge': 0, 'multiplicity': 1}, 4: {'atoms': [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)], 'charge': 0, 'multiplicity': 1}, 5: {'atoms': [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754

In [9]:
# let's MOVE 2 units on xy axes the first molecule (I) and rotate 180 deg around z-axis the second molecule (water)
w5_I_moved_rotated = w5_I.translate(0, x=2, y=2, z=0).rotate(1, x=0, y=0, z=180)

print(w5_I_moved_rotated.xyz)

	16
-- charge= 0 and multiplicity= 1 --
I     	     2.00000000	     2.00000000	     0.00000000
O     	    -0.00000000	     0.16954767	     0.00000000
H     	    -0.58708000	    -0.58799233	     0.00000000
H     	     0.58708000	    -0.58799233	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000



### IMPORTANT FACTS:
- After translating a molecule, a new object is created where the moved 
molecule in the same position. The same is true for any rotation
- Rotation are around the center of mass of the target molecule

In [10]:
# let's visualize its random process FREEZING Iodine
from copy import deepcopy

# creating a brand new copy when 'I' will NOT be moved or rotated 
w5 = deepcopy(w5_I)

# setting a sphere radius for the Cluster spherical boundary conditions
w5.sphere_radius = 10

!pip install py3Dmol
import py3Dmol

In [12]:
from numpy import random
random_gen = random.default_rng(1234)

# to save snapshot and show as a movie
w5_xyz = ""

total_steps = 10000

for i in range(total_steps):
    # molecule to be selected between [0, w.total_molecules]
    mol = random_gen.choice(w5.total_molecules)

    # angle between [0, 360)
    ax = random_gen.uniform() * 360
    ay = random_gen.uniform() * 360
    az = random_gen.uniform() * 360

    # moving between [-move, +move]
    move = w5.sphere_radius / 2
    tx = move * (random_gen.uniform() - 0.5)
    ty = move * (random_gen.uniform() - 0.5)
    tz = move * (random_gen.uniform() - 0.5)

    # saving coordinates as a string
    # w5_xyz += w5.xyz
    if i % 100 == 0: 
        w5_xyz += w5.xyz

    w5 = w5.translate(mol, x=tx, y=ty, z=tz).rotate(
        mol, x=ax, y=ay, z=az
    )
    # printing current step
    print(
        f"\r progress {100.0*((i + 1)/total_steps):.2f}"
        f" % -- step {i + 1}/{total_steps}", end=''
        )

# -------------------------------------------------------
print("\n *** JOB DONE ***")
print(f"after {total_steps} steps\n")
print(w5.xyz)

 progress 100.00 % -- step 10000/10000
 *** JOB DONE ***
after 10000 steps

	16
-- charge= 0 and multiplicity= 1 --
I     	     1.24330557	    -3.85808240	    -3.32292499
O     	    -0.61791256	    -5.76133972	     4.79515226
H     	    -0.97798416	    -6.54417063	     5.21474013
H     	    -1.33124684	    -5.43188935	     5.34392036
O     	     4.75992181	     7.86546699	    -1.36707428
H     	     4.84817344	     7.70046654	    -0.42711882
H     	     3.89257951	     7.56167538	    -1.09512252
O     	    -8.10386629	     3.05078217	     2.28520988
H     	    -8.65202307	     3.66080722	     1.78930329
H     	    -8.07784005	     3.86297286	     2.79334244
O     	    -0.48463868	    -0.58830024	     8.75383894
H     	    -0.09466273	    -0.42698312	     9.61431864
H     	    -0.87880536	     0.23555642	     9.04439462
O     	     8.90264528	    -2.70065381	    -3.80135572
H     	     8.26165977	    -3.39641692	    -3.64779449
H     	     8.78824260	    -2.71469398	    -2.84991225



In [13]:
# py3Dmol: a simple IPython/Jupyter widget to embed an interactive 3Dmol.js viewer in a notebook.
!pip install py3Dmol
import py3Dmol

In [15]:
# 20% to check visually that everything is inside
sr = w5.sphere_radius * 1.2

# sphere center
cx, cy, cz = w5.sphere_center
sc = {"x": cx, "y": cy, "z": cz}

# axes
x_axis = {'start': {'x': cx-sr, 'y':cy, 'z':cz}, 'end': {'x': cx+sr, 'y':cy, 'z':cz}}
y_axis = {'start': {'x': cx, 'y':cy-sr, 'z':cz}, 'end': {'x': cx, 'y':cy+sr, 'z':cz}}
z_axis = {'start': {'x': cx, 'y':cy, 'z':cz-sr}, 'end': {'x': cx, 'y':cy, 'z':cz+sr}}

# starting visualization
xyz_view = py3Dmol.view(width=700, height=500)#, linked=False, viewergrid=(2,2))
xyz_view.addModelsAsFrames(w5_xyz,'xyz')
# xyz_view.setStyle({'stick': {}})
xyz_view.setStyle({'sphere': {'radius': 0.8}})
xyz_view.addSphere({'center': sc, 
        'radius': sr, 
        'color' :'yellow',
        'alpha': 0.5,
        })

# animation base on several XYZ coordinates snapshots
xyz_view.animate({'loop': "forward", 'speed': 1, 'reps': 1})

# cartesian 3D axes
xyz_view.addLine(x_axis)
xyz_view.addLine(y_axis)
xyz_view.addLine(z_axis)

xyz_view.addLabel("x", {
        'position':x_axis["end"],
        'inFront':'true',
        'fontSize':20,
        'showBackground':'false',
        'fontColor': 'black',
        })
xyz_view.addLabel("y", {
        'position':y_axis["end"],
        'inFront':'true',
        'fontSize':20,
        'showBackground':'false',
        'fontColor': 'black',
        })
xyz_view.addLabel("z", {
        'position':z_axis["end"],
        'inFront':'true',
        'fontSize':20,
        'showBackground':'false',
        'fontColor': 'black',
        })

xyz_view.zoomTo()
xyz_view.show()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol